In [1]:
import keras

In [2]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150,150,3))

In [3]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = "C:/Users/yang/Desktop/nongzuowu/small/"

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir,'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i*batch_size:(i+1)*batch_size] = features_batch
        labels[i*batch_size:(i+1)*batch_size] = labels_batch
        i+=1
        if i * batch_size>=sample_count:
            break
    return features, labels

In [4]:
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [7]:
conv_base.trainable = False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_datagen = ImageDataGenerator(
               rescale=1./255,
               rotation_range=40,
               width_shift_range=0.2,
               height_shift_range=0.2,
               shear_range=0.2,
               horizontal_flip=0.2,
               fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(150,150),
            batch_size=20,
            class_mode='binary')

validation_genetator = test_datagen.flow_from_directory(
            validation_dir,
            target_size=(150,150),
            batch_size = 20,
            class_mode = 'binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(train_generator, 
                              steps_per_epoch=100, 
                              epochs=30,
                              validation_data=validation_genetator,
                              validation_steps=50,
                              )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
100/100 [==============================] - 44s 440ms/step - loss: 0.6110 - acc: 0.6865 - val_loss: 0.5327 - val_acc: 0.7710
Epoch 2/30
100/100 [==============================] - 44s 441ms/step - loss: 0.4964 - acc: 0.7855 - val_loss: 0.4740 - val_acc: 0.7940
Epoch 3/30
100/100 [==============================] - 44s 440ms/step - loss: 0.4501 - acc: 0.8085 - val_loss: 0.4494 - val_acc: 0.8020
Epoch 4/30
100/100 [==============================] - 44s 441ms/step - loss: 0.4299 - acc: 0.8135 - val_loss: 0.4309 - val_acc: 0.8030
Epoch 5/30
100/100 [==============================] - 44s 443ms/step - loss: 0.4153 - acc: 0.8155 - val_loss: 0.4160 - val_acc: 0.8110
Epoch 6/30
100/100 [==============================] - 44s 444ms/step - loss: 0.4045 - acc: 0.8200 - val_loss: 0.4089 - val_acc: 0.8200
Epoch 7/30
100/100 [==========